This code varies the design parameters for the single qubit-resonator example and varies them according to a gaussian distribution with a set percentage of std. deviation. The later part of the codes then plots the results.

# Basic packages

In [ ]:
%load_ext autoreload
%autoreload 2
import time

In [ ]:
import json
import design as d
import names as n

with open("design_variables.json") as in_file:
    initial_design_variables = json.load(in_file)

from qdesignoptimizer.utils.chip_generation import create_chip_base, ChipType
from qdesignoptimizer.utils.utils import close_ansys
close_ansys()

In [ ]:
# for this example to store the data in the right place
import os
os.makedirs(os.path.dirname("out/"), exist_ok=True)

## Design assembly

In [ ]:
CHIP_NAME = "multi_transmon_chip"
OPEN_GUI = True
chip_type = ChipType(size_x="10mm", size_y="10mm", size_z="-300um")
design, gui = create_chip_base(
    chip_name=CHIP_NAME, chip_type=chip_type, open_gui=OPEN_GUI
)

n.add_design_variables_to_design(design, initial_design_variables)

In [ ]:
def render_qiskit_metal_design(design, gui):
    d.add_transmon_plus_resonator(design, group=n.GROUP_1)
    d.add_transmon_plus_resonator(design, group=n.GROUP_2)

    d.add_coupler(design)

    d.add_route_interconnects(design)

    d.add_launch_pads(design)

    d.add_chargeline(design, group=n.GROUP_1)
    d.add_chargeline(design, group=n.GROUP_2)

    gui.rebuild()
    gui.autoscale()


render_qiskit_metal_design(design, gui)

## MiniStudy

In [1]:
import mini_studies as ms
import optimization_targets as ot
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qdesignoptimizer.design_analysis_types import MeshingMap

In [ ]:
def CoupledLineTee_mesh_names(comp_names):
    all_names_to_mesh = [f"prime_cpw_{comp_names}", f"second_cpw_{comp_names}"]
    return all_names_to_mesh

In [ ]:
MINI_STUDY_GROUP = n.GROUP_1
MINI_STUDY = ms.get_mini_study_qb_res(group=MINI_STUDY_GROUP)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    groups=[MINI_STUDY_GROUP],
    opt_target_qubit_freq=True,
    opt_target_qubit_anharm=True,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=True,
    opt_target_resonator_qubit_chi=True,
)

# Definition of randomization in design parameters

In [ ]:
import numpy as np
from copy import deepcopy
from qdesignoptimizer.utils.utils import get_value_and_unit

percentage_variation = 20
no_optimization_runs = 5

randomized_design_variables = deepcopy(initial_design_variables)

# Optimization runs

In [ ]:
start_time = None
end_time = None
for optimization_run in range(no_optimization_runs):
    if optimization_run == 0:
        start_time = time.strftime("%Y%m%d-%H%M%S")

    for opt_target in opt_targets:
        value, unit = get_value_and_unit(initial_design_variables[opt_target.design_var])
        randomized_design_variables[opt_target.design_var] = f"{np.random.normal(value, value*percentage_variation/100)}{unit}"

    n.add_design_variables_to_design(design, randomized_design_variables)


    design_analysis_state = DesignAnalysisState(
        design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
    )
    design_analysis = DesignAnalysis(
        design_analysis_state,
        mini_study=MINI_STUDY,
        opt_targets=opt_targets,
        save_path="out/" + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
        update_design_variables=False,
        plot_settings=ps.PLOT_SETTINGS,
        meshing_map=[
            MeshingMap(component_class=CoupledLineTee, mesh_names=CoupledLineTee_mesh_names)
        ],
    )

    group_runs = 10  # 10
    group_passes = 8  # 6
    delta_f = 0.001
    for i in range(group_runs):
        design_analysis.update_nbr_passes(group_passes)
        design_analysis.update_delta_f(delta_f)
        design_analysis.optimize_target({}, {})
        design_analysis.screenshot(gui=gui, run=i)

    if optimization_run == no_optimization_runs-1:
        end_time = time.strftime("%Y%m%d-%H%M%S")

In [ ]:
print(start_time)
print(end_time)

# Plotting results

In [ ]:
from qdesignoptimizer.utils.plotting import load_data_by_date, plot_optimization_results

files = load_data_by_date('out', CHIP_NAME, start_time, end_time)
plot_optimization_results(files, plot_variance= False,plot_type='variable_vs_iteration',opt_target_list=opt_targets,plot_design_variables_sorted=False)

In [ ]:
close_ansys()